In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
exp_name = '2020_12_18_stepper'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total.tail()

# Preprocessing

## a. compute distance estimates

TODO: this part is hacky and will be improved by getting imu estimates rather than flow dx/dy estimates, which turn out to be very unreliable.

In [ ]:
from wall_analysis import add_distance_estimates

VELOCITY = 0.05 # [m/s], parameter in crazyflie
D_START = 0.6 # [m], starting distance 

df_total = df_total.assign(d_estimate=None) # initialize df_estimate
df_total = df_total.apply(add_distance_estimates, axis=1)

for i, row in df_total.iterrows():
    
    plt.figure()
    df_total.iloc[i] = add_distance_estimates(row, ax=plt.gca())
    #plt.plot(row.yaw_deg)u
    if row.z is not None:
        plt.plot(row.seconds, row.z, color='C2')
    plt.xlabel('time [s]')
    plt.ylabel('z [mm]')
    #plt.ylim(-90, 90)
    plt.ylim(0, 650)
    #plt.xlim(MIN_S, MAX_S)
    
    fig, ax = plt.subplots()
    ax.plot(row.positions[:, 0], label='x', color='C0')
    ax.plot(row.positions[:, 1], label='y', color='C1')
    ax.plot(row.positions[:, 2], label='z', color='C2')
    #ax.plot(row.dx)
    #ax.plot(row.dy)
    ax.set_ylim(-10, 50)
    #ax.set_xlim(0, 1000)
    ax.set_xlabel('position idx [-]')
    ax.set_ylabel('x, y [mm]')
    ax.legend(loc='lower left')
    ax.set_title("lateral movement, " + row.source + row.appendix)

## b. compute spectrogram

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS
from wall_analysis import add_spectrogram
all_frequencies = np.ceil(np.fft.rfftfreq(N_BUFFER, 1/FS))
df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

# Analysis

## 1. SNR study

In [ ]:
def plot_spectrogram(ax, row, min_freq, max_freq, mic_idx=0, **kwargs):
    plot_freq = (all_frequencies > min_freq) & (all_frequencies < max_freq)
    plot_times = ~np.isnan(row.d_estimate)
    assert np.any(plot_times)
    #print('time window with valid distances:', np.where(plot_times)[0][[0, -1]])
    spec = row.spectrogram[plot_freq, :, mic_idx][:-1, :]
    spec = spec[:, plot_times][:, :-1]
    spec[spec > 0] = np.log10(spec[spec > 0])
    spec[spec == 0] = np.nan
    
    if row.distance == -51:
        d_increasing = row.d_estimate[plot_times][::-1]
        spec_increasing = spec[:, ::-1]
    else:
        d_increasing = row.d_estimate[plot_times]
        spec_increasing = spec
    ax.pcolorfast(d_increasing, all_frequencies[plot_freq], spec_increasing, **kwargs)

def plot_total(ax, row, min_freq, max_freq, mic_idx=0, method=np.nanmean, **kwargs):
    plot_freq = (all_frequencies > min_freq) & (all_frequencies < max_freq)
    plot_times = ~np.isnan(row.d_estimate)
    spec = row.spectrogram[plot_freq, :, mic_idx]
    spec[np.isnan(spec)] = 0.0
    assert np.all(spec >= 0), spec[np.where(spec<0)]
    spec = spec[:, plot_times]
    spec[spec == 0] = np.nan
    total = method(spec, axis=1)
    freq = all_frequencies[plot_freq]
    ax.plot(total, freq, **kwargs)
    ax.set_xlabel(method.__name__ + ' of spectrogram')

In [ ]:
print(df_total.appendix.unique())
print(df_total.source.unique())
print(df_total.distance.unique())
#appendix = "_new"
#source = "mono3125"

In [ ]:
from wall_analysis import filter_by_dicts


min_freq = 100
max_freq = 5000
mic_idx = 0

filter_dict = [{"mic_type":mic_type}]
dfs = filter_by_dicts(df_total, filter_dict)

dfs = dfs.sort_values(by=['motors', 'source'], axis=0)
#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for motors, df_motors in dfs.groupby('motors'):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    i_col = 0
    for i, row in df_motors.iterrows():
        source = row.source
        if source in ["mono1750", "mono3875", "sweep_fast"]:
            continue
        i_col += 1
        fig, axs = plt.subplots(1, 2, squeeze=False, sharey=True)
        fig.set_size_inches(10, 5)
        plot_spectrogram(axs[0, 0], row, min_freq, max_freq, mic_idx=mic_idx)
        plot_total(axs[0, 1], row, min_freq, max_freq, mic_idx=0, method=np.nanmean, color=f"C{i_col}")
        #axs[0, 0].set_xlabel(f'seconds [s]')
        axs[0, 0].set_xlabel(f'distance [m]')
        axs[0, 0].set_ylabel('frequency [Hz]')
        fig.suptitle(f'spectrogram of mic{mic_idx}, source {source}, motors {row.motors}, appendix {row.appendix}')
        #axs[0, 1].set_xlim([0, maxi])

        plot_total(ax, row, min_freq, max_freq, mic_idx=0, method=np.nanmean, label=source, color=f"C{i_col}")
        if "mono" in source:
            freq = int(source.strip("mono"))
            ax.axhline(freq, color=f"C{i_col}", ls=":")
            axs[0, 1].axhline(freq, color=f"C{i_col}", ls=":")
    #ax.set_xscale('log')
    ax.set_ylabel('frequency [Hz]')
    ax.legend()
    ax.set_title(f'motors: {motors}')

## 2. Propeller noise study

In [ ]:
import scipy.signal

appendix = ""
source = "None"
mic_type = "audio_deck"

filter_dict = {"source":source, "appendix":appendix, "mic_type":mic_type, "distance":51}
df_sources = filter_by_dicts(df_total, [filter_dict])
assert len(df_sources) == 1, df_sources
row = df_sources.iloc[0]


plt.figure()
times_idx = ~np.isnan(row.d_estimate)
spec = row.spectrogram[:, times_idx, :]
    
mask = (all_frequencies > min_freq) & (all_frequencies < max_freq)
spec[spec==0] = np.nan
spec_average = np.nanmean(spec[mask, :, :], axis=(1, 2))

max_indices = scipy.signal.find_peaks(spec_average, height=5)[0][:3]
#max_indices = np.argsort(spec_average)[::-1][:3]
plt.plot(all_frequencies[mask], spec_average)
[plt.axvline(all_frequencies[mask][m], color=f'C{j}', label=f'peak{j}', ls=':') for j, m in enumerate(max_indices)]
plt.grid(which='both')
plt.title('total spectrogram, average of 4 mics')
print('max frequencies:', all_frequencies[mask][max_indices])

In [ ]:
#fig, axs = plt.subplots(len(max_indices), n_mics, sharex=True, squeeze=False, sharey=True)

n_harmonics = 4
fig, axs = plt.subplots(n_harmonics, n_mics, sharex=True, squeeze=False)#, sharey=True)
fig.set_size_inches(20, 10)

# choose only times where the max snr bin is given. 
#freqs = all_frequencies[mask][max_indices]
freq_max = all_frequencies[mask][max_indices][1] 
times_idx = np.where(row.frequencies_matrix[:, 1] == freq_max)[0]
if not len(times_idx):
    print(f'never have {freq_max} as strongest index: {row.frequencies_matrix[:, 1]}')
distances = row.d_estimate[times_idx] * 100

#for j, f_idx in enumerate(max_indices):
for j in range(n_harmonics):
    freq = freq_max * (j + 1)
    print(freq)
    f_idx = np.where(all_frequencies[mask] == freq)[0][0]
    #freq = int([f_idx])
    spec = row.spectrogram[mask, :, :]
    
    for i in range(n_mics):
        spec_here = spec[f_idx, times_idx, i]
        #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
        axs[j, i].plot(distances, spec_here, color=f'C{j}', label=f'{freq:.0f}Hz')
        axs[j, i].legend(loc='upper left')
        axs[j, i].set_title(f'mic{i}')
    #[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]
    #[axs[0, i].set_xlim(5, 60) for i in range(n_mics)]
    [axs[-1, i].set_xlabel('distance [cm]') for i in range(n_mics)]
    pass

## 1. frequency selection

Investigate 
- what bins are chosen by forced scheme (black dots below)
- when spectrogram at given bin is nonzero 

Above suggests that even when we do not play the given bins, there is significant propeller noise there (thus they are chosen among the 32 strongest frequency bins). The only exception is bin 2375, which is very clean.

In [ ]:
print('available sources:', df_total.source.unique())
filter_dict = {
    'source': 'None',
    #'source': 'mono1750',
    #'source': 'mono2375',
    #'source': 'mono3125',
    #'source': 'mono3875',
    #'source': 'sweep_slow',
    #'source': 'sweep_fast',
    'appendix': '_new'
}
rows = filter_by_dicts(df_total, [filter_dict])
assert len(rows) == 1
row = rows.iloc[0]

In [ ]:
for source in df_total.source.unique():
    #if not 'mono'in source:
        #continue
    filter_dict['source'] = source
    rows = filter_by_dicts(df_total, [filter_dict])
    assert len(rows) == 1
    row = rows.iloc[0]
    
    for mic_idx in [2]:
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)

        plot_spectrogram(ax, row, min_freq=100, max_freq=5000, mic_idx=mic_idx)
        [ax.axhline(f, label=f'{f}', color=f'C{i}', ls=':') for i, f in enumerate(FREQS)]
        ax.set_title(row.source + row.appendix + f" mic{mic_idx} snr:{row.snr}")
        ax.legend(loc='lower left')

In [ ]:
print(filter_dict)
rows = filter_by_dicts(df_total, [filter_dict])
assert len(rows) == 1
row = rows.iloc[0]

strongest_bin = row.frequencies_matrix[:, 0]
fig = plt.figure()
fig.set_size_inches(10, 5)
plt.scatter(row.seconds, strongest_bin, color='k', label='forced')

#mic_indices = [0]
mic_indices = range(n_mics)

for i, f in enumerate(FREQS):
    plt.axhline(f, label=f'{f}', color=f'C{i}')
    
    # find out where the spectrogram is non-zero for this frequency bin. 
    f_idx = np.where(f == all_frequencies)[0][0]
    
    spec_total = np.mean(row.spectrogram[f_idx, :, mic_indices], axis=0)
    spec_total -= np.min(spec_total)
    spec_total *= 50 #/ np.max(spec_total)
    times = row.seconds[spec_total > 0]
    plt.scatter(times, f + spec_total[spec_total > 0], color=f'C{i}')
    plt.legend()
    if len(times) > 0:
        plt.xlim(times[0], 18)
    #plt.xlim(times[0], times[-1])

## 2. interference studies

In [ ]:
appendix = "_new"

freqs = FREQS

fig, axs = plt.subplots(len(freqs), n_mics, sharex=True)#, sharey=True)
fig.set_size_inches(20, 20)
for j, f in enumerate(freqs): 
    f_idx = np.where(f == all_frequencies)[0][0]
    
    source = f"mono{f}"
    df_sources = filter_by_dicts(df_total, [{"source":source, "appendix":appendix}])
    assert len(df_sources) == 1
    row = df_sources.iloc[0]
    
    # forced
    #times_idx = np.where(f == row.frequencies_matrix[:, 0])[0]
    # automatic
    #times_idx = np.where(np.sum(spectrogram[f_idx, :, :], axis=1) > 0)[0]
    times_idx = ~np.isnan(row.d_estimate)
    distances = row.d_estimate[times_idx] * 100
    
    for i in range(n_mics):
        spec = row.spectrogram[f_idx, times_idx, i]
        #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
        axs[j, i].plot(distances, spec, color=f'C{j}', label=f'{f} Hz')
        axs[j, i].legend(loc='upper left')
        axs[j, i].set_title(f'mic{i}')
#[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]
#[axs[0, i].set_xlim(5, 60) for i in range(n_mics)]
[axs[-1, i].set_xlabel('distance [cm]') for i in range(n_mics)]
pass

In [ ]:
freqs = FREQS

sources = ['sweep_fast', 'sweep_slow']
appendix = "_new"
for source in sources:
    
    df_sources = filter_by_dicts(df_total, [{"source":source, "appendix":appendix}])
    for i, row in df_sources.iterrows():
        
        fig, axs = plt.subplots(len(freqs), n_mics, sharex=True)#, sharey=True)
        fig.set_size_inches(20, 20)
        fig.suptitle(row.source + row.appendix, y=0.9)
        for j, f in enumerate(freqs): 
            f_idx = np.where(f == all_frequencies)[0][0]

            # forced
            times_idx = np.where(f == row.frequencies_matrix[:, 0])[0]
            # automatic
            #times_idx = np.where(np.sum(spectrogram[f_idx, :, :], axis=1) > 0)[0]
            #times_idx = ~np.isnan(row.d_estimate)
            distances = row.d_estimate[times_idx] * 100

            for i in range(n_mics):
                spec = row.spectrogram[f_idx, times_idx, i]
                #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
                axs[j, i].plot(distances, spec, color=f'C{j}', label=f'{f}', marker='x')
                axs[j, i].legend(loc='upper left')
                axs[j, i].set_title(f'mic{i}')
        #[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]